In [1]:
import json

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
with open('./dataset/blass.json', 'r') as f:
    blass = json.load(f)

In [5]:
with open('./dataset/bronze.json', 'r') as f:
    bronze = json.load(f)

In [6]:
all_bronze_materials = []

for item in bronze['colorings']:
    for r in item['recipes']:
        for m in r['materials']:
            all_bronze_materials.append(m['name'])

In [7]:
all_bronze = pd.DataFrame(all_bronze_materials, columns=['name']).groupby('name').agg({'name': 'count'})

In [8]:
all_blass_materials = []
for item in blass['colorings']:
    for r in item['recipes']:
        for m in r['materials']:
            all_blass_materials.append(m['name'])

In [9]:
all_blass = pd.DataFrame(all_blass_materials, columns=['name']).groupby('name').agg({'name': 'count'})

In [10]:
all_bronze = all_bronze.rename(columns={'name': 'count'}).reset_index()

In [11]:
all_blass = all_blass.rename(columns={'name': 'count'}).reset_index()

In [12]:
all_blass.shape

(88, 2)

In [13]:
all_bronze.shape

(87, 2)

In [14]:
all_material = list(all_blass['name']) +list(all_bronze['name'])

In [15]:
for_col_mat = pd.DataFrame(all_material, columns=['name']).groupby('name').agg({'name': 'count'}).rename(columns={'name': 'count'}).reset_index()

In [16]:
all_methods_bronze = []
for item in bronze['colorings']:
    for r in item['recipes']:
        all_methods_bronze.append(r['method']['name'])

In [17]:
all_methods_blass = []
for item in blass['colorings']:
    for r in item['recipes']:
        all_methods_blass.append(r['method']['name'])

In [18]:
all_methods = all_methods_bronze + all_methods_blass

In [19]:
for_col_method =  pd.DataFrame(all_methods, columns=['name']).groupby('name').agg({'name': 'count'}).rename(columns={'name': 'count'}).reset_index()

In [20]:
cols = ['id', 'type'] + list(for_col_mat['name']) + list(for_col_method['name'])

In [21]:
def cols_to_id(key):
    for i in range(len(cols)):
        if cols[i] == key:
            return i

In [22]:
def feature_columns(item):
    columns = [0 for i in range(len(cols))]
    
    # ID
    columns[0] = item['id']
    
    # bronze or blass
    if item['id'] >= 1000 and item['id'] < 2000:
        columns[1] = 1
    else:
        columns[1] = 2
    
    for r in item['recipes']:
        # material
        for m in r['materials']:
            columns[cols_to_id(m['name'])] = m['amount']
        
        # method
        columns[cols_to_id(r['method']['name'])] = 1
    
    return columns

In [23]:
features = []

for item in bronze['colorings']:
    features.append(feature_columns(item))

    
for item in blass['colorings']:
    features.append(feature_columns(item))


In [24]:
feature_df = pd.DataFrame(features, columns=cols)

In [25]:
json_str = []
for item in bronze['colorings']:
    data = {}
    data['0. id'] = item['id']
    data['1. title'] = item['title']
    data['2. type'] = 'bronze'
    for m in all_material:
        data['5. ' + m] = 0
    
    data['3. num_methods'] = len(item['recipes'])
    
    for i, r in enumerate(item['recipes']):
        for m in r['materials']:
            data['5. ' + m['name']] = m['amount']
        
        data['4. method_{0}'.format(i)] = r['method']['name']
        
    json_str.append(data)

    
for item in blass['colorings']:
    data = {}
    data['0. id'] = item['id']
    data['1. title'] = item['title']
    data['2. type'] = 'blass'
    for m in all_material:
        data['5. ' + m] = 0
    
    data['3. num_methods'] = len(item['recipes'])
    
    for i, r in enumerate(item['recipes']):
        for m in r['materials']:
            data['5. ' + m['name']] = m['amount']
        
        data['4. method_{0}'.format(i)] = r['method']['name']
    
    json_str.append(data)

In [26]:
from PIL import Image

import glob

image_list = glob.glob('../create_dataset/dataset/images/*@0x0.png')

image_dic = {}
for image in image_list:
    item_id = image.split('/')[-1].split('@')[0]
    image_dic[item_id] = image

In [27]:
canvas = Image.new('RGB', (12800, 640), (0,0,0))

for i, item in enumerate( json_str ):
    item_id = str(item['0. id'])

    if image_dic.get(item_id) is not None:
        img = Image.open(image_dic[item_id])
        img = img.resize((128, 128))
        
        h = int(i / 100) * 128
        w = int(i % 100) * 128
        canvas.paste(img, (w, h))

canvas.save('./atlas.png')

atlas_url = './atlas.png'

In [28]:
# Display the Dive visualization for this data
from IPython.core.display import display, HTML

HTML_TEMPLATE = """<link rel="import" href="/nbextensions/facets-dist/facets-jupyter.html">
        <facets-dive id="elem" height="600"
                sprite-image-width="128"
                sprite-image-height="128">
        </facets-dive>
        <script>
          var data = {jsonstr};
          var atlas_url = "{atlas_url}";
          document.querySelector("#elem").data = data;
          document.querySelector("#elem").atlasUrl = atlas_url;
        </script>"""
html = HTML_TEMPLATE.format(jsonstr=json_str, atlas_url=atlas_url)
display(HTML(html))